<a href="https://colab.research.google.com/github/William-Schwarz/Decifra-Contrato/blob/william_ds/Decifra_Contrato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:

!pip -q install google-genai google-adk PyPDF2 python-docx

In [146]:
import os
from google.api_core import exceptions
from google.colab import userdata
from google import genai
from google.colab import files
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
import PyPDF2

warnings.filterwarnings("ignore")

In [147]:
# Configura a API Key do Google Gemini
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
except Exception as e:
    print(f"⚠️ Erro ao configurar a chave da API do Google Gemini: {e}")
    exit()

In [148]:
# Configura o cliente da SDK do Gemini
try:
    client = genai.Client()
    MODEL_ID = "gemini-2.0-flash"
except Exception as e:
    print(f"⚠️ Erro ao inicializar o cliente Gemini: {e}")
    exit()

In [149]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    try:
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
                        final_response += "\n"
    except exceptions.ServiceUnavailable as e:
        return f"⚠️ Serviço indisponível ao comunicar com o agente '{agent.name}'. Por favor, tente novamente mais tarde. Detalhes: {e}"
    except exceptions.InvalidArgument as e:
        return f"⚠️ Argumento inválido fornecido ao agente '{agent.name}'. Verifique a entrada. Detalhes: {e}"
    except Exception as e:
        return f"⚠️ Erro ao comunicar com o agente '{agent.name}': {e}"
    return final_response

In [150]:
# Função auxiliar que faz a leitura do arquivo
def ler_arquivo(nome_arquivo):
    try:
        if nome_arquivo.endswith(".pdf"):
            with open(nome_arquivo, 'rb') as file:
                try:
                    reader = PyPDF2.PdfReader(file)
                    text = ""
                    for page_num in range(len(reader.pages)):
                        page = reader.pages[page_num]
                        text += page.extract_text()
                    return text
                except Exception as e:
                    return f"⚠️ Erro ao ler o conteúdo do arquivo PDF '{nome_arquivo}': {e}"
        elif nome_arquivo.endswith(".docx"):
            try:
                from docx import Document
                document = Document(nome_arquivo)
                text = ""
                for paragraph in document.paragraphs:
                    text += paragraph.text + "\n"
                return text
            except ImportError:
                return "⚠️ A biblioteca 'python-docx' não está instalada. Por favor, instale-a com: pip install python-docx"
            except Exception as e:
                return f"⚠️ Erro ao ler o conteúdo do arquivo DOCX '{nome_arquivo}': {e}"
        elif nome_arquivo.endswith(".txt"):
            with open(nome_arquivo, 'r', encoding='utf-8') as file:
                return file.read()
        else:
            return "⚠️ Formato de arquivo não suportado. Por favor, envie um arquivo .pdf, .docx ou .txt."
    except FileNotFoundError:
        return f"⚠️ Erro: Arquivo '{nome_arquivo}' não encontrado."
    except Exception as e:
        return f"⚠️ Erro ao processar o arquivo '{nome_arquivo}': {e}"

In [151]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [152]:
# Agente 1: Receptor e Extrator de Contrato
def agente_receptor():
    receptor = Agent(
        name="agente_receptor",
        model=MODEL_ID,
        instruction="""
        Você é o agente inicial do sistema Decifra-Contrato. Sua tarefa é receber o contrato fornecido pelo usuário.
        Este contrato pode ser um texto diretamente inserido ou o conteúdo de um arquivo.

        Uma vez que o contrato é fornecido, sua principal função é extrair o texto completo e limpo do contrato.
        Remova qualquer formatação desnecessária que possa dificultar a análise posterior (como quebras de linha excessivas, numerações automáticas, etc.).

        Se o usuário fornecer informações adicionais sobre o tipo de contrato (por exemplo, "contrato de aluguel", "termo de serviço"), anote essa informação para auxiliar os próximos agentes.

        Ao final, entregue o texto puro do contrato para o próximo agente.
        """,
        description="Agente que recebe e extrai o texto do contrato."
    )
    return receptor

In [153]:
# Agente 2: Identificador de Jargões
def agente_identificador():
    identificador = Agent(
        name="agente_identificador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em identificar jargões jurídicos e termos técnicos dentro de um texto de contrato.
        Sua tarefa é analisar o texto do contrato fornecido e listar todos os termos que provavelmente seriam desconhecidos por uma pessoa leiga.

        Considere termos como: latim jurídico (e.g., 'ad hoc'), termos técnicos específicos da área do contrato (e.g., em um contrato de tecnologia, 'API', 'SaaS'), e cláusulas com redação complexa.

        Apresente a lista de jargões identificados de forma clara e concisa para o próximo agente.
        """,
        description="Agente que identifica jargões jurídicos e técnicos."
    )
    return identificador

In [154]:
# Agente 3: Explicador de Termos
def agente_explicador(google_search):
    explicador = Agent(
        name="agente_explicador",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em explicar termos jurídicos e técnicos de forma clara e acessível para pessoas leigas.
        Você receberá uma lista de jargões identificados em um contrato.

        Para cada termo da lista, sua tarefa é fornecer uma explicação concisa e em linguagem simples.
        Se necessário, utilize a ferramenta de busca do Google (google_search) para encontrar definições e explicações adequadas.

        Tente fornecer exemplos práticos ou analogias para facilitar a compreensão.

        Apresente cada termo seguido de sua explicação de forma clara e organizada.
        """,
        description="Agente que explica jargões jurídicos e técnicos para leigos.",
        tools=[google_search]
    )
    return explicador

In [155]:
# Agente 4: Resumidor e Destacador de Pontos Chave
def agente_resumidor():
    resumidor = Agent(
        name="agente_resumidor",
        model=MODEL_ID,
        instruction="""
        Você é um especialista em resumir contratos e destacar os pontos mais importantes para uma pessoa leiga.
        Com base no texto completo do contrato, sua tarefa é gerar um resumo conciso, com linguagem simples e direta.

        Além do resumo, identifique e liste as cláusulas ou pontos que são cruciais para o entendimento do usuário, como:
        - Objeto do contrato
        - Obrigações das partes
        - Prazos e condições
        - Formas de pagamento (se aplicável)
        - Condições de rescisão
        - Penalidades (se houver)

        Apresente o resumo e os pontos chave de forma clara e organizada, utilizando marcadores ou listas para facilitar a leitura.
        """,
        description="Agente que resume contratos e destaca pontos chave."
    )
    return resumidor

In [164]:
print("📜🧐 Iniciando o Sistema Decifra-Contrato (com suporte a arquivos) 📜🧐")

contrato_texto = ""
while True:
    opcao = input("❓ Deseja colar o texto do contrato (digite 'texto') ou carregar um arquivo (digite 'arquivo')? ").lower()
    if opcao in ['texto', 'arquivo']:
        break
    else:
        print("\n⚠️ Opção inválida. Por favor, digite 'texto' ou 'arquivo'.")

if opcao == 'texto':
    contrato_texto = input("Cole o texto do contrato aqui: ")
elif opcao == 'arquivo':
    try:
        uploaded = files.upload()
        if uploaded:
            nome_arquivo = list(uploaded.keys())[0]
            contrato_texto = ler_arquivo(nome_arquivo)
            if isinstance(contrato_texto, str) and contrato_texto.startswith("⚠️"):
                print(f"\n{contrato_texto}")
                contrato_texto = ""
            else:
                print(f"\nArquivo '{nome_arquivo}' lido com sucesso.")
        else:
            print("\n⚠️ Nenhum arquivo foi carregado.")
    except Exception as e:
        print(f"\n⚠️ Ocorreu um erro ao carregar o arquivo: {e}")
        contrato_texto = ""

# Executa o pipeline, independente da origem do texto
if contrato_texto:
    try:
        receptor = agente_receptor()
        identificador = agente_identificador()
        explicador = agente_explicador(google_search)
        resumidor = agente_resumidor()
    except Exception as e:
        print(f"⚠️ Erro ao inicializar os agentes: {e}")
        contrato_texto = ""

if contrato_texto:
    display(Markdown("## ✨ Iniciando processamento..."))

    display(Markdown("### ⏳ Etapa 1: Extraindo texto do contrato..."))
    texto_contrato_extraido = call_agent(receptor, contrato_texto)
    if texto_contrato_extraido.startswith("⚠️"):
        display(Markdown(f"### ⚠️ Extração: <span style='color:orange'>{texto_contrato_extraido}</span>"))
    else:
        display(Markdown("### 📝 Contrato Extraído"))
        display(to_markdown(texto_contrato_extraido))

        display(Markdown("### ⏳ Etapa 2: Identificando jargões..."))
        jargoes_identificados = call_agent(identificador, texto_contrato_extraido)
        display(Markdown("### 🔎 Jargões identificados"))
        display(to_markdown(jargoes_identificados))

        display(Markdown("### ⏳ Etapa 3: Explicando termos..."))
        explicacoes_termos = call_agent(explicador, jargoes_identificados)
        display(Markdown("### 💡 Termos explicados"))
        display(to_markdown(explicacoes_termos))

        display(Markdown("### ⏳ Etapa 4: Resumindo contrato e destacando pontos chave..."))
        resumo_contrato = call_agent(resumidor, texto_contrato_extraido)
        display(Markdown("### 📚 Resumo gerado"))
        display(to_markdown(resumo_contrato))

        display(Markdown("## ✅ Análise Concluída!"))

📜🧐 Iniciando o Sistema Decifra-Contrato (com suporte a arquivos) 📜🧐
❓ Deseja colar o texto do contrato (digite 'texto') ou carregar um arquivo (digite 'arquivo')? arquivo


Saving Modelo-CONTRATO-DE-PRESTAÇÃO-DE-SERVIÇOS.pdf to Modelo-CONTRATO-DE-PRESTAÇÃO-DE-SERVIÇOS (11).pdf

Arquivo 'Modelo-CONTRATO-DE-PRESTAÇÃO-DE-SERVIÇOS (11).pdf' lido com sucesso.


## ✨ Iniciando processamento...

### ⏳ Etapa 1: Extraindo texto do contrato...

### 📝 Contrato Extraído

...

> Ok, entendi! Vou explicar cada um desses jargões e termos técnicos de forma clara e acessível:
> 
> 
> Aqui estão as explicações dos termos:
> 
> *   **Contratante/Contratado:** O **contratante** é a pessoa ou empresa que está pagando por um serviço ou produto, ou seja, quem contrata. O **contratado** é quem está prestando o serviço ou fornecendo o produto. Em vez desses termos, você pode usar "empresa" e "cliente", "prestador de serviços" e "contratante" ou os nomes das partes envolvidas.
> 
> *   **Obra audiovisual cinematográfica:** Refere-se a uma produção que utiliza imagens em movimento e som, criada para ser exibida em salas de cinema. Inclui filmes de ficção, documentários e animações.
> 
> *   **Roteiro e a concepção artística:** O **roteiro** é o texto que descreve a história, os diálogos e as cenas do filme. A **concepção artística** é a visão criativa geral do filme, incluindo o estilo visual, a atmosfera e a mensagem que o diretor e a equipe querem transmitir.
> 
> *   **Técnico cinematográfico:** É o profissional que trabalha nos bastidores da produção de um filme, utilizando equipamentos e conhecimentos técnicos para garantir a qualidade do som, da imagem, da iluminação e de outros aspectos técnicos.
> 
> *   **Função de (preencher):** Refere-se ao cargo ou papel específico que o contratado irá desempenhar na produção do filme. É importante detalhar as responsabilidades e tarefas associadas a essa função.
> 
> *   **Obrigações inerentes:** São as responsabilidades que fazem parte da função que o contratado vai exercer. Por exemplo, se o contratado for um editor de vídeo, uma obrigação inerente é entregar o material editado no prazo combinado.
> 
> *   **Prazo indeterminado:** Significa que o contrato não tem uma data de término definida. Ele continua valendo até que uma das partes decida encerrá-lo.
> 
> *   **De forma irretratável e irrenunciável:** Essa cláusula significa que o contratado não pode cancelar ou desistir do que foi acordado. Uma vez assinado, o contrato deve ser cumprido até o fim, a menos que haja um acordo entre as partes.
> 
> *   **Direitos patrimoniais e conexos:** Os **direitos patrimoniais** são os direitos de explorar economicamente a obra, como vendê-la, licenciá-la ou reproduzi-la. Os **direitos conexos** são os direitos de outros profissionais que contribuíram para a obra, como atores, músicos e técnicos.
> 
> *   **Meios audiovisuais competentes existentes e a serem criados:** Essa expressão garante que os direitos de exibição e distribuição do filme valem para todas as tecnologias atuais e futuras, como cinema, televisão, internet, DVDs, Blu-rays e outras que venham a surgir.
> 
> *   **Legislação vigente:** Refere-se às leis que se aplicam ao contrato. No entanto, é recomendável especificar quais leis são relevantes, como a Lei de Direitos Autorais, a Lei do Audiovisual e o Código Civil.
> 
> *   **Extinção do presente contrato, em qualquer uma das formas:** Refere-se às diferentes maneiras pelas quais o contrato pode ser finalizado, como por exemplo, término do prazo, cumprimento de todas as obrigações, acordo entre as partes (rescisão amigável) ou descumprimento do contrato por uma das partes.
> 
> *   **Direitos de cunho moral:** São os direitos do autor de uma obra de ser reconhecido como o criador dela e de ter sua obra protegida contra alterações ou usos que possam prejudicar sua reputação. Esses direitos não podem ser vendidos ou renunciados.
> 
> *   **Vigência:** É o período de tempo em que o contrato é válido e produz efeitos. A data de início e término da vigência devem estar claramente especificadas no contrato.
> 
> *   **Instrumento:** É o próprio contrato. Em vez de "instrumento", você pode simplesmente usar a palavra "contrato".
> 
> *   **Totalidade dos entendimentos:** Essa cláusula confirma que o contrato representa o acordo completo entre as partes, ou seja, tudo o que foi combinado está escrito no contrato e nada mais.
> 
> *   **De igual teor e forma:** Significa que todas as cópias do contrato são idênticas, com o mesmo conteúdo e aparência. Isso garante que não haja dúvidas sobre qual é a versão original do contrato.
> 


### ⏳ Etapa 4: Resumindo contrato e destacando pontos chave...

### 📚 Resumo gerado

> ## Resumo do Contrato de Prestação de Serviços (Curta-Metragem)
> 
> Este contrato estabelece os termos do acordo entre o CONTRATANTE (diretor do curta-metragem) e o CONTRATADO (técnico cinematográfico) para a realização do curta-metragem "_____________________". O CONTRATADO prestará serviços técnicos na função de _____________________________________, comprometendo-se a seguir o cronograma e as orientações da direção. Em contrapartida, o CONTRATADO cede os direitos patrimoniais e conexos de seu trabalho ao CONTRATANTE, sem receber remuneração financeira. O CONTRATADO terá seu nome creditado na obra. O contrato é válido até a finalização do curta-metragem.
> 
> ## Pontos Chave do Contrato:
> 
> *   **Objeto do Contrato:** Prestação de serviços técnicos cinematográficos pelo CONTRATADO na função de _____________________________________ para o curta-metragem "_____________________".
> *   **Obrigações do CONTRATADO:**
>     *   Realizar os serviços técnicos conforme o cronograma e as orientações da direção.
>     *   Ceder os direitos patrimoniais e conexos de seu trabalho ao CONTRATANTE.
> *   **Obrigações do CONTRATANTE:**
>     *   Creditar o nome do CONTRATADO na obra.
> *   **Prazos e Condições:** O contrato tem validade a partir da assinatura até a finalização da obra, com os serviços a serem realizados no período de ______________.
> *   **Forma de Pagamento:** Não há remuneração financeira para o CONTRATADO.
> *   **Condições de Rescisão:** Em caso de extinção do contrato, o CONTRATANTE poderá utilizar os trabalhos já realizados e os direitos autorais cedidos.
> *   **Penalidades:** O contrato não menciona penalidades específicas.


## ✅ Análise Concluída!